In [1]:
import os
import time
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

path = os.getcwd()

df_dir = path + "/dfs"
temp_dir = path + "/dfs/temp"

options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": temp_dir,
    "download.prompt_for_download": False,
    "profile.default_content_setting_values.automatic_downloads": 1
}

options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)


url = "https://www3.bcb.gov.br/ifdata/legado.html"
driver.get(url)

wait = WebDriverWait(driver, 30)

In [2]:
anomes = driver.find_elements(By.CSS_SELECTOR, "#ulDataBase li a")
anomes_lista = [a.get_attribute("href") for a in anomes]
anomes_nome  = [a.get_attribute("textContent").strip() for a in anomes]

In [3]:
def wait_new_download(before_set, timeout=60):
    t0 = time.time()
    while time.time() - t0 < timeout:
        files = set(os.listdir(path + "/dfs/temp/"))
        new = list(files - before_set)

        # ignore partial downloads
        new_done = [f for f in new if f.endswith(".csv")]

        if new_done:
            # pick newest by mtime
            new_done_paths = [os.path.join(path + "/dfs/temp/", f) for f in new_done]
            return max(new_done_paths, key=os.path.getmtime)

        time.sleep(0.25)

    raise TimeoutError("Download did not finish / no new file appeared")

In [5]:
for a, n in zip(anomes_lista, anomes_nome):
    
    driver.execute_script(a)
    
    relatorio = driver.find_elements(By.CSS_SELECTOR, "#ulRelatorio li a")
    relatorio_lista = [r.get_attribute("href") for r in relatorio]
    relatorio_nome = [r.get_attribute("textContent").strip() for r in relatorio]
    
    for r, r_n in zip(relatorio_lista, relatorio_nome):
        
        driver.execute_script(r)
        
        before = set(os.listdir(path + "/dfs/temp/"))
        

        export = wait.until(EC.element_to_be_clickable((By.ID, "aExportCsv")))
        ActionChains(driver).move_to_element(export).pause(0.1).click(export).perform()
                
        downloaded_path = wait_new_download(before, timeout=60)
        
        n = n.replace("/", "")
        r_n = r_n.replace(" ", "_")
        
        final_name = f"{n}_{r_n}.csv"
        final_path = os.path.join(df_dir, final_name)

        shutil.move(downloaded_path, final_path)
        
driver.close()